In [4]:
import ee
import xee
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xee
import cartopy.feature
import xarray as xr


# Initialize the Earth Engine module
ee.Initialize()

def fetch_and_process_era5(start_date, end_date, country_name='United Kingdom'):
    # Load the country boundaries
    countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
    country = countries.filter(ee.Filter.eq('country_co', 'UK'))

    # Load ERA5 daily dataset and clip it to the country boundary
    era5_daily = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterDate(start_date, end_date) \
        .filterBounds(country) 
        
    # Define the conditions for skiable days
    wind_speed_threshold_mps = 50 / 2.23694  # Convert mph to m/s
    snow_depth_threshold_m = 0.25

    # Map function to check both conditions
    def check_conditions(image):
        # Calculate resultant wind speed
        u_wind = image.select('u_component_of_wind_10m')
        v_wind = image.select('v_component_of_wind_10m')
        resultant_wind_speed = u_wind.pow(2).add(v_wind.pow(2)).sqrt()

        # Check wind and snow conditions
        suitable_wind = resultant_wind_speed.lt(wind_speed_threshold_mps)
        suitable_snow = image.select('snow_depth').gt(snow_depth_threshold_m)

        # mask image with conditions
        # suitable_wind = suitable_wind.updateMask(suitable_wind)
        # suitable_snow = suitable_snow.updateMask(suitable_snow)

        # masked_image = image.updateMask(suitable_wind.And(suitable_snow))

        # create boolean image
        # skiable_day = suitable_wind.And(suitable_snow)
        # skiable_day    
        
        # Return image with combined conditions
        return suitable_wind.And(suitable_snow).rename('skiable_day')

    # Apply the map function
    skiable_days = era5_daily.map(lambda image: check_conditions(image))

    #sum all images in the collection
    skiable_days = skiable_days.select('skiable_day')
    print('skiable_days', skiable_days.getInfo())

    total_skiable_days = skiable_days.sum().rename('total_skiable_days')
    total_skiable_days = total_skiable_days.clip(country)
    
    # Sum the skiable days
    # total_skiable_days = skiable_days.reduce(ee.Reducer.sum()).rename('total_skiable_days')
    # total_skiable_days = total_skiable_days.select('total_skiable_days').clip(area)
    total_skiable_days = ee.ImageCollection(total_skiable_days)

    # clip image to the country boundary
    # total_skiable_days = total_skiable_days.map(lambda image: image.clip(area))

    # Convert to xarray Dataset
    ds = xr.open_dataset(total_skiable_days, engine='ee')#, crs='EPSG:4326', scale=0.25)
    
    return ds

# Define parameters
start_date = '1950-01-01'
end_date = '1955-12-31'

# Fetch and process data
ds = fetch_and_process_era5(start_date, end_date, 'United Kingdom')



skiable_days {'type': 'ImageCollection', 'bands': [], 'version': 1731645754825311, 'id': 'ECMWF/ERA5_LAND/DAILY_AGGR', 'features': [{'type': 'Image', 'bands': [{'id': 'skiable_day', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}], 'properties': {'system:index': '19500102'}}, {'type': 'Image', 'bands': [{'id': 'skiable_day', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}], 'properties': {'system:index': '19500103'}}, {'type': 'Image', 'bands': [{'id': 'skiable_day', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}], 'properties': {'system:index': '19500104'}}, {'type': 'Image', 'bands': [{'id': 'skiable_day', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 

c:\Users\s1834371\Miniconda3\envs\xee\Lib\site-packages\xee\ext.py:551: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


In [6]:
ds

<xarray.Dataset>
Dimensions:             (time: 1, lon: 360, lat: 180)
Coordinates:
  * time                (time) int32 0
  * lon                 (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * lat                 (lat) float32 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
Data variables:
    total_skiable_days  (time, lon, lat) int32 ...
Attributes:
    crs:      EPSG:4326

In [ ]:
# def plot_skiable_days_xarray(ds):
#     # Plot using Matplotlib and Cartopy
#     fig, ax = plt.subplots(figsize=(10, 12), subplot_kw={'projection': ccrs.PlateCarree()})
#     ax.set_extent([-10, 2, 50, 60], crs=ccrs.PlateCarree())
    
#     ax.add_feature(cartopy.feature.LAND, edgecolor='black')
#     ax.add_feature(cartopy.feature.OCEAN)
#     ax.add_feature(cartopy.feature.RIVERS)
#     ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
#     ax.coastlines(resolution='10m')
    
#     # Plot the data
#     im = ax.imshow(ds.total_skiable_days.values, extent=[-10, 2, 50, 60], transform=ccrs.PlateCarree(), cmap='viridis', origin='upper')
#     cbar = fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.036, pad=0.04)
#     cbar.set_label('Total Skiable Days')
    
#     plt.title('Skiable Days in the UK (1950-1955)')
#     plt.show()

# plot_skiable_days_xarray(ds)
